### OGC WFS

OGC Web Feature Service deals for downloading of vector data. Usually OGC GML format is used for data exchange.

#### Searching for the data in CSW server

NOTE: You can skip this step and use the URL directly

First step: let's find some WFS service in our CSW server:

In [ ]:
from owslib.fes import PropertyIsLike, BBox, And, PropertyIsEqualTo
from owslib.csw import CatalogueServiceWeb

thecsw = CatalogueServiceWeb('http://geoportal.gov.cz/php/micka/csw/index.php')

wfs_query = PropertyIsLike('csw:AnyText', 'WFS')
geology_query = PropertyIsLike('csw:AnyText', 'Geology')
service_query = PropertyIsLike('apiso:type', 'service')
geology_and_wfs = And([geology_query, wfs_query, service_query])
thecsw.getrecords2([geology_and_wfs], esn='full')

print(thecsw.results)

for recid in thecsw.records:
    record = thecsw.records[recid]
    print(recid, record.title)

Let's have a look at WFS data in czech Geology survey

In [ ]:
geology = thecsw.records['575a4ef6-2f74-43ed-9352-6f400a010852']
print("Abstract:\n %s\n" % geology.abstract)

print("Identifier: %s" % geology.identifiers[1]['identifier'])

#### Download data using Web Feature Service
Let's read some data

In [ ]:
from owslib import wfs as webfeatureservice
url = "http://inspire.geology.cz/geoserver/wms?service=WMS&version=1.3.0&request=Getcapabilities"
geology_wfs = webfeatureservice.WebFeatureService(geology.identifiers[1]['identifier'])



Service metadta (Capabilities):

In [ ]:
capabilities = geology_wfs.getcapabilities()
print("URL: %s" % capabilities.geturl())
print("Name: %s" % geology_wfs.provider.name)
print("Title: %s" % geology_wfs.identification.title)
print("Keywords: %s" % geology_wfs.identification.keywords[0])
print("Fees: %s" % geology_wfs.identification.fees)
print("Abstract: %s" % geology_wfs.identification.abstract)

Print list of available layers 

In [ ]:
for i in geology_wfs.contents:
    print("\n#### {} ####".format(i))
    print(geology_wfs.contents[i].abstract)

Download data from selected layer

In [ ]:
identifier = 'gsmlp:CZE_CGS_500k_Fault'
features = geology_wfs.getfeature([identifier])

with open("../outputs/geology_faults.gml", "w") as out:
    out.write(features.read())



[Download](../outputs/geology_faults.gml) and open the data using QGIS.

![](images/geology_faults.png)

---

[<- WMS](03_wms.ipynb) | [Working with projections ->](../05_projections/01_projections.ipynb)